# Introduction to Spatial Analysis

The outcome of the practical this week will seem oddly familiar to students taking the QGIS intensive later in the term because we’re going to start with similar data and have a similar objective in mind; however, we’re going to be tackling the problem in an entirely different, more geocomputational manner. In short, rather than doing everything in the Model Builder we’re going to be using code. 

The purpose of the practical is to identify feasible sites for wind turbines based on a number of selection criteria; these are:
* Site must not be located within 2,000m of a populated area; 
* Site must be within 3,000m of a major road;
* Site must not be located within 250m of a river;
* Site must be on land where the slope is less than 17 degrees in steepness;
* Site must be West-facing in orientation (including SW and NW).

This analysis will require moving between vector and raster formats, using Boolean algebra, and automation.


## Setting Up

Unzip the Practical-8-data zipfile into a folder that is easy to access – **WARNING**: we are working with a lot of data this week. Do not try to store all of the intermediate and source shapefiles in Dropbox!

### Previewing in QGIS

Let's preview the data in QGIS -- that's a bit easier than simply loading data blindly into geopandas or PySAL:
* Using the ‘Add vector layer’ button, add the shapefiles for Sites, for LandUse, for Rivers, and for A- and M-Roads.
* For the land use layer you’ll simply classify land with a random colour scheme (or you can specify one if you like) using the ALC_GRADE attribute column. 
* Now click on the Raster menu, then under ‘Miscellaneous’ pick ‘Build Virtual Raster (Catalog)’. You should be able to make sense of the settings here, but in short: tell it to load the entire LiDAR directory for Sussex and save your catalog with a useful name like ‘Sussex DTM’.

When you get to near the end of the practical, you will need to download the remaining geodata for Week 8. All of the data used in this practical is available as free, open data.

# Vector Analysis

The order of the requirements above was _not_ random: the first three requirements are all based on vector analyses; the last two are both based on raster analyses. It makes sense to do them separately and then _combine_ the analysis at the end.

## Requirement 1: Distance from Population

Since we’ve given you the data, let’s start by conducting a site analysis in Sussex; however, note that the whole point of this approach is that once we’ve worked things out there, we’ll be able to do the same for the other sites fairly quickly!

### Extracting Data using Geography

To keep things simple, we’re going to use the ‘Urban’ classification from the Land Use data as a proxy for ‘populated area’. So what obviously need here is a 2,000m buffer around urban areas. Let’s begin by just trimming the Urban class to the size of the site since that’s the extent of the area that we care about.

Click on the ‘Processing’ menu and then pick ‘Graphical Modeler’ to open up the model builder toolkit. It should look something like Figure 1 below.

Note that there are three areas to the modelling toolbox: on the left-hand side you have both Inputs and Algorithms, and the right-hand side you have the area where you build the model and also the model’s name and group.
So for this first, simple model you might want give it the name ‘Cookie Cutter’ and the group ‘Practical 8’.
Now we need to start adding elements to the model, but let’s think this through: we want to cut down the Land Use layer so that it only contains the area falling inside the chosen site; then we want to build a buffer around just the urban part of that layer, and we want to save this as ‘Population Buffer’ for the site. The Graphical Modeller can do that so that you can easily repeat this operation for every other site as well!

You can change the description to ‘Select Site’ if you like to make it clear what this step is doing, but here’s what the rest of the options mean:
* Input Layer: the vector layer from which we want to select by attribute.
* Selection attribute: the column in which we want to look for the match, which in this case (if you followed my instructions in 2.5) is site_nm.
* Operator: we’re looking for an exact match on a text string so ‘=’ is the way to go.
* Value: the thing we’re going to match on, which you set by clicking on the drop-down button at the right side and then selecting the Site Name parameter!
Once this is all set up click ‘OK’ and you should see a change in the model to something like Figure 5 below in which two of the inputs are now connected to an algorithm by grey lines pointing to the ‘In’ side of the algorithm.

Why did we use the Site Name parameter instead of just hard-coding the word ‘Sussex’ into the model?
Our next step is to select the urban areas from the Land Use data set – happily, this is also a ‘Select by attribute’ operation so we just double-click the algorithm on the left-hand side panel to add another box to the model. This time there’s no screen-shot, but you should be able to work out the parameters from the clues below:
* The Input Layer is going to be the land use data.
* We’re looking in the ALC_GRADE column for a way to select populated areas.
* We’re looking for areas whose class is ‘Urban’ but here’s one important hint: you do not need to put single- or double-quotes around the Value!

Once this is all set up and you’ve clicked ‘OK’, you should see a new algorithm that is connected only to the Land Use Layer by a grey line. That’s part two done!

The last step is to bring the outputs of these two operations together so that we can perform a Boolean operation that will cut away any part of the Urban layer that falls outside of the site area and then return that as our result. For this you can use either the ‘Clip’ or the ‘Intersect’ algorithms. The are both under (again) QGIS Geoalgorithms, and it seems like the Intersect algorithm is slightly more reliable on GeoCUP, though in the screenshots below I’ve used Clip since I was doing work on a Mac and had no problems – functionally, they do the same thing.

The parameters are shown in Figure 6, and note how I’ve changed the description so that it’s clear what is going on. But more importantly, notice how the ‘Input layer’ and ‘Clip layer’ are not shapefiles, but have been changed to use the output of the two earlier algorithms!

The last thing to note is that there’s a parameter called ‘X<OutputVector>’ and that I’ve filled this in as ‘Urban Areas within Site’ – this tells the model builder that I want QGIS to create an output – in this case, it’s a temporary file – at this point in the model. As we’ll see later, that doesn’t have to mean that the model is finished, but it will add a new temporary layer to QGIS so that we can see the results of the process.


### Adding Buffering

Ooops, it turns out that we forgot to do the buffering part! Let’s go back and add this to the model now:
Right-click on the Cookie Cutter model and select ‘Edit model’.

This will bring up the model Process modeller window and your graphical model.

Now we want to add a buffering algorithm so let’s do another search for available options…

As always, when given the option I prefer to use the algorithms offered by QGIS itself since they’re more likely to do sensible things with projections and other inputs that some of the ‘algos’ offered by GDAL/OGR or Grass GIS.
Here’s how I’d set the ‘Fixed distance buffer’ geoalgorithm up to work:
* Change the description to ‘Add Urban Buffer’.
* The ‘Input layer’ is going to be the output from the clipping algorithm.
* The distance is the one specified by the requirements provided at the top of this file.
* Dissolve result should be ‘Yes’.
* Let’s call the output ‘Urban Areas Buffer’.

When you click ‘OK’, you’ll see that there are now two grey lines running out of the clipping algorithm – one to the teal output file, and one to the ‘Add Urban Buffer’ box – and there’s another grey line running from the buffer to a new teal output file called ‘Urban Areas Buffer’.

We don’t really need the earlier ‘Urban Areas within Site’ output any more, so let’s get rid of it by clicking on the ‘pencil’ icon at the lower-right corner of the clipping algorithm to bring up the edit dialogue.

All you need to do is delete the text ‘Urban Areas within Site’ from the OutputVector option and then hit ‘OK’ to save your changes – the earlier output should now be gone.

Save the changes to the model and run it again for Sussex, perhaps saving the output this time to a new shapefile called ‘Sussex Urban Buffer’.

Do the same for the other two sites using the same model – this should require you to change exactly one input parameter.

Now might also be a good time to rename your model to ‘Urban Analysis’.

Remember to save your model when you’re done!


## Requirement 2: Maximum Distance from Main Roads

You are aiming for the model shown in Figure 10. This obviously looks quite complex on first glance, but based on everything you’ve learned about programming you should eventually be able to make sense of it by stepping through each part in turn – the fact that the model is graphical should make this even easier than if you were stepping through code!

The parts of the model are set out below.

The first stage of the model is the same as for the Cookie Cutter model that we created to buffer the urban areas: we have a ‘Site Name’ string input that the user will specify (and we set it to default to ‘Sussex’) and a Sites layer that we’ll be looking in for a match against the site_nm attribute.

Then, we add two new inputs: these are obviously vector layers and they are also obviously lines. We call one A Roads and the other M Roads so that it’s easy to know which layers you should be picking when the model runs. Note how naming your inputs and your model steps makes it much easier to know what you have to connect.

Then we clip each of these in turn to the ‘Select Site’ so that we’re only looking at road areas that fall within the site area. Technically, an area could be accessible from a road that falls outside of the site but we’re taking a simplistic approach here so let this go for now.

Once these layers have been clipped, we then merge them so that they are in a single layer, simplify them (I used 100m as the threshold) to reduce their complexity, dissolve them (since we don’t care if they’re A- or M-roads, we just want a single buffer), and then create the buffer of 3,000m around them.

You’ll notice that I have two outputs from this model: one after the dissolve, and one after the buffer. I do this so that I can check that the algorithms up to the buffering step have done what I expected. The result of having two outputs is that two layers will be added to the Layers Panel when the algorithm runs.

Again, the whole point of this is that we can now perform the same analysis for any site simply by launching a model, selecting the parameters, and then allowing it to run. We know that we’ll get consistent results (we won’t accidentally forget to include the A-Roads or something) for each site and we know that if we realise we’ve made a mistake (should have included B-Roads as well!) then we could just update the model and re-run it for each site… that’s obviously not quite as elegant as having a for loop in a Python script and re-running the script once, but it’s not half-bad considering…

To help you out, here are the key parameters for each of the new algorithms:
* For clipping the road layers (the Clip geoalgorithm), although it should be easy to remember what the ‘Input layer’ should be for each, remember that the ‘Clip layer’ is going to be the output of the selection function on the site layer, and not the site layer itself. You will leave Clipped<OutputVector> blank because we don’t want an output going into the Layer Panel from this operation.
* For Merge vector layers, remember that the two inputs layers are the outputs of the two clipping functions – that’s why we give them different descriptions: so that you can tell which is which when you’re adding them to the merge algorithm.
* For Simplify geometries, your input is the output of the merge operation, and I picked a tolerance of 100.0 (metres) on the basis that that would reduce the complexity of the layer without really impacting the buffer (which is 3km!).
* For dissolving the simplified, merged roads, we want to set the Dissolve geoalgorithm to ‘Dissolve all’, so set this to ‘Yes’ as the separate roads will have all been merged into a single multi-line record. Note that here we do have an output – Accessible Roads – that we can use as a ‘sanity check’ of our processing. Once we’re happy that this model works as we want it to, we could always just remove the output by deleting the text string from the Dissolved<OutputVector> option.
* Finally, for the Fixed distance buffer, we just set the distance to 3,000.0 (metres) and then specify an output so that our analysis is saved as a new (temporary or permanent) layer.

Remember to save your model when you’re done!


## Requirement 3: River Flooding Exclusion Zones 

The river buffer zone (which is ultimately about exclusion not accessibility, as was the case with the roads) is conceptually very similar to the Road Analysis model just created. It is probably faster to build it from scratch, but you might find it easier to try to save a copy of your Roads Analysis model as River Analysis and then edit from there (remember to change the inputs and descriptions so that the model makes sense to someone else). Be warned: I’m not entirely certain the ‘Save As’ always created a duplicate of a model – I’d suggest creating from scratch!

Since the process is so similar, I’d like you to figure out this model for yourself although I’ll include the graphical view below in Figure 11 – remember, all we’re trying to do at this point is build the inputs to the final stage of the modelling process in which we’ll start to build a better understanding of feasible sites using a kind of ‘sieve’ analysis.


## Combining the Vector Layers

Making allowances for colours, after running the three vector-based analyses above you should have something like the following on your QGIS map. In Figure 12 the brown areas are the river buffer (so we can’t build there), the dark grey-green areas are the urban buffer (so we can’t build there either), and the dark blue areas are the roads buffer (so we must build within that area).

Let’s now combine these three layers into a single result: the areas that are safe, unobtrusive, and accessible. The model for this is shown in Figure 13.

There are just two new geoalgorithms to learn for this model:

The Difference algorithm will (as we saw in class) basically remove those bits of the Input layer that overlap areas of the Difference layer. Just two inputs, so it’s pretty straightforward: all we need to do to remove both the urban and the river areas is to run the difference algorithm twice, using the output of the first as the input to the second.

The Reproject layer algorithm may be needed (it was with some of my earliest tests) to set the right CRS for the output of the difference operation. You should by this point have practically memorised the EPSG number needed for most spatial analysis in the UK.

If all goes well with this final vector algorithm, then you should end up with the following (see brown areas in Figure 14) as the potential areas within which we can search for sites that are (in theory at least) safe, unobtrusive, and accessible.

Remember to save your model when you’re done!


# Raster Analysis

Our ultimate goal is to produce three rasters from the input DTM, and we could actually do all of this in one model and get the results that we need from combining the Slope and Aspect analyses. However, I’ll leave that as an ‘advanced’ exercise for someone who wants to stretch their powers of attention and in this practical we’re going to tackle these as separate problems and, consequently, separate models:
* A Slope Analysis raster that gives us a 1/0 output depending on whether the slope is less than 17 degrees.
* An Aspect Analysis raster that gives us a 1/0 output depending on whether the aspect of a slope faces West, Southwest, or Northwest (which is the prevailing wind direction).
* A Slope Aspect raster that gives us values in the range 10 to 88 as a function of both slope and aspect, and which allows us to make a nice-looking map that shows two variables changing at once: more information about the Aspect-Slope map (designed by Cynthia Brewer, one of the leading thinkers of colour in cartography) can be found here: https://blogs.esri.com/esri/arcgis/2008/05/23/aspect-slope-map/. Note: we’re only going to do this bit at the end, after you’ve got the final analysis model working!

As I said, technically we could actually do all of these analyses in one go, but it’s a bit easier to break it apart into smaller chunks.

## Requirement 4: Site Must not be on a Steep Slope

For the slope analysis, we want to find areas of the DTM where the slope is less than 17 degrees. The idea behind this constraint that you don’t want to build a wind farm on the side of the cliff, or on steep hill where access is going to be an issue.

Let’s talk out what we need to do:
1. First, we want to ensure that the DTM is in the right projection – this is just a safety check in case there’s a missing projection file.
2. Then we’ll want to perform the slope analysis itself to calculate degree values for each raster cell.
3. After that, we want to use Boolean logic to convert the cells to 1/0 depending on whether they are less than 17 degrees in steepness, or more.
4. Finally, although it shouldn’t be necessary you might need to add another reprojection just to ensure it’s referenced correctly. I got mixed results here: it seems like some of the algorithms lose the projection information so you end up needing to manually specify it… unless you add this final step. See what you get!

We don’t need to save any of the intermediate steps, just save the output as a Slope Analysis. Figure 15 is what your model should look like at the end.

You might have noticed that this time we don’t have a ‘Site’ input – that’s because we (as far as I can tell) have no way to use that parameter input to select a layer by name (e.g. `Find the layer named <Site>_DTM`). Luckily, as long as you’ve named your rasters something sensible it’s fairly easy to pick out each site’s DTM. 

By way of strategy, from this point on I’d suggest completing the analysis for one site and then moving on to the next one. That way, when you’re done with each site you can delete all of the temporary layers and you won’t end up with three "Slope Analysis" layers in your Layer Panel with no idea which one to pick when running the final model for each site!

You can see in the model shown in Figure 15 that we are now using algorithms from sources other than QGIS. The little 'planet with beanie' icon means it's from GDAL/OGR, and the grass icon means that the algorithm is from GRASS GIS. I’m not going to give you too many pointers at this stage since you should be becoming familiar with the process, but here are the key things you need to know:

* The slope algorithm is called r.slope and, once you’ve specified the input as the result of the warp function, you should be able to just leave the other parameters ‘as is’.
* The ‘Find acceptable slopes’ function comes from r.mapcalculator. You will only be specifying Raster Layer A (which is the output of the slope calculation) and a formula (the rest of the parameters can be left ‘as is’). 

The formula works is as follows (pay attention as you’ll need to work out the formula for the aspect analysis): 
`1*(A <= 17.0)`
Remember that I said logical tests can return 1/0 and that means the same as True/False? Here, `A` means ‘Raster Layer A’ and what we’re doing is testing the value of each cell against the number in the logical test. So another way of thinking about this is: `for each cell in the raster, is its value less than or equal to 17 degrees?`

After that it’s just a case of either outputting the result or, if you get a projection problem, adding the warp function again and saving the result as the `Slope Analysis`.

Remember to save your model when you’re done!

## Requirement 5: Site Must be West-Facing

The aspect analysis, as I said above, looks at which direction a cell in the raster faces. These can be understood by looking at Figure 16. As you can see, the only slightly tricky thing is that ‘North’ wraps around from 337.5 to 22.5 degrees.

The model for the aspect analysis is necessarily going to look almost exactly the same as that of the slope analysis in Figure 15. The steps in this process are:
* Warp (reproject) into EPSG:27700.
* Calculate the aspect (r.aspect from the GRASS GIS algorithms).
* Find the acceptable aspects (r.mapcalculator) using a formula.
* Warp (reproject) again if necessary.
* Save the output as a temporary layer called ‘Aspect Analysis’.

The formula for the aspect analysis is ever so slightly more complex, but let’s think it through: we are looking for parts of ‘A’ that are greater than or equal to 202.5 (Southwest) and that these parts of A also have to be less than 337.5 degrees (North). The key thing you need to know to make this formula work is that the syntax for ‘AND’ here is “&&” (two ampersands).

**WARNING**: although it would appear that you can specify the formula in the GRASS raster calculator as 202.5 <= RasterA@1 < 337.5, I would be wary of using this syntax as there are very few programming languages that recognise it.

For consistency, use the approach I’ve suggested in 3.2.2.

You can find a full list of the operators in GRASS (and a lot more besides) here: https://grass.osgeo.org/grass70/manuals/r.mapcalc.html 

I’m not going to give you a screenshot of the model as you should be able to work it out from the description above. 

Remember to save your model when you’re done!


# Combining Raster and Vector Analysis

OK, so we’ve got a polygon that tells us where we should look for feasible sites: ones that are safe (won’t get flooded), accessible (not too far from a main road), and acceptable (not too near urban areas… we don’t care about the farmers!). And we’ve got two rasters with Boolean 1/0 values that tell us which areas have appropriate terrain characteristics: not too steep, and generally facing the right direction for prevailing wind. Now we can put this all together to generate a final, polygonal result.

Here’s what we’re going to do:
* We’re going to trim each raster using the acceptable areas polygon (there’s no point doing this analysis in areas that won’t work).
* We’re going to then combine the two rasters such that cells in both layers have to have a 1 for the area to be acceptable – think about this: it falls inside the areas we’ve defined as acceptable, it has a slope that is less than or equal to 17 degrees, and it has a westerly aspect. Any other combination is ruled out mathematically using this simple logic: 1*1=1; 1*0=0; 0*1=0; and 0*0=0. Hint: this is pointing the formula that you’ll need for the r.mapcalculator algorithm.
* We’ll then ‘polygonise’ (convert back to a vector) the results of the raster calculation – this will have a column called ‘Class’ that has a 0 or 1 value depending on the results of the previous calculator function.

After that, we select only those areas that have a 1 (because they conform to all of our requirements).
And then we’ll modify the selection so that only those polygons whose areas are greater than 500m2 are retained (there’s no point keeping tiny sliver polygons).

And we save that output as the ‘Result’!
 
Figure 17 shows what that looks like in Model Builder.

The inputs should be fairly obvious now: the ‘Slope Analysis’ and ‘Aspect Analysis’ inputs are the rasters that we created in the two earlier steps; the ‘Acceptable Areas’ input is polygon that we derived from the various buffers created before that. See how we can use the Acceptable Areas input twice?

I’m going to focus here on the new bits:
* Note that there are two clipping operations – so different inputs for each one, and the ‘Mask layer’ is going to be the ‘Acceptable Areas’ polygon (which is why it’s connected to the clipping algorithm).
* ‘Combine Rasters’ is another r.mapcalculator operation which has as inputs the output from the clipping operations and uses a very simple formula that combines layers A and B such that each must be ‘True’ for the result to be ‘True’.
* Make sure that you make a note of the ‘Output field name’ in the polygonise step as you’ll need that field name later!

Reprojection should be unnecessary, so you might try building the model without it as you can always add it later (by simply changing the input for the next step: ‘Select by attribute’ to rewire the model).

‘Select by Attribute’ will need the name of the field that you created at the polygonise step, and we want to select all areas whose value is 1 (meaning that the input rasters both had Ones).

We’ll then select by expression and here you need to know two things: first, if you’ve ever dug around in the Field Calculator, you’ll have noticed that ‘$area’ is the area of a feature in a polygon layer; and second, we want to remove from our previous selection those areas whose area is less than 500 (i.e. 500m2). With those two pieces of information you should be able to figure out what to type into the ‘Expression’ input and what to specify under ‘Modify current selection by…’

Finally, we want to ‘Save selected features’ so that we retain only those polygons which are acceptable by all of the criteria established earlier and which have an area greater than or equal to 500m2.

RESULT!

Just as an aside, you’ll also see in Figure 17 that I output a copy of the raster – this is just a ‘sanity check’ to make sure that what comes out of the polygonisation and last few steps lines up with what came out of the raster analysis. It also allows me to compare the results of that analysis with the input rasters that I was using. You could output the results from each and every step, but it is more useful to have outputs where they’ll help you to check that the model did as you expected and that the results make sense.

If all has gone well, then Figure 18, Figure 19, and Figure 20 should match what you got as the possible locations for wind farms in each of the selected site areas. Of course, this is a fairly simplistic analysis since there are a number of things we’ve not taken into account: are they overly visible (do you really want to put them on top of the highest hill in the area? If not, how might you go about removing areas that are too prominent?); are they inside an AONB or SSSI (both of which would probably be no-nos from a regulatory standpoint); and so on. 

So we could continue to add constraints to the model until the cows come home, but at this point you should understand enough about how these work to be able to build your own for your research!

There is also nothing stopping you from combining all of the models created above into a single monster-model that would basically run from start to finish with minimal human interaction. However, it’s worth keeping in mind the trade-offs between growing power and growing complexity – as your model gets bigger and bigger, you’re more likely to screw up a step and have trouble with the outputs. Debugging these big models is also harder. I haven’t really looked into this, but the ‘Batch Process’ option might be the way to go here as it suggests that you can create a ‘batch’ of jobs (i.e. models) and run them all at once.

Remember to save your model!


# Obtaining the Geodata
There you go! You’ve got a model that, given the same inputs for some other site, can now exactly replicate your analysis. In fact, you could even muck about with the QGIS style sheets so that they are automatically styled the same way! With that kind of replication in mind, you could now download the data for the other sites and perform the same analysis.

## Obtaining the DTM data
We get the Digital Terrain Model data from DEFRA’s open LiDAR web site.

For example, for the Lake District area, we would want [tile SD49](http://environment.data.gov.uk/ds/survey/index.jsp#/survey?grid=SD49).

The 2m resolution LiDAR data set is high-resolution enough for our purposes (it should be the first one that you see below the map) and you could download and save it to the appropriate subfolder in your working directory.

Unzip it but do not try add each of the .asc files in the unzipped LiDAR directory separately.
Switching to QGIS, click on the Raster menu, then under ‘Miscellaneous’ pick Build Virtual Raster (Catalog). You should be able to make sense of the settings here, but in short: tell it to load the entire LiDAR directory and save your catalog with a useful name like ‘Lake District DTM’.

Another area of interest could be, say, [Dartmoor](http://environment.data.gov.uk/ds/survey/index.jsp#/survey?grid=SX59).

## Obtaining OS OpenData

We can get the data on rivers from the Ordnance Survey OpenData products: https://www.ordnancesurvey.co.uk/business-and-government/products/os-open-rivers.html.

You’ll need to click through a few pages to get to the actual download page, but the advantage of that is that you can get the OpenRoads data at the same time.

The OpenRoads file is massive, so let’s do a little quick preparation… Select the roads layer and, using the ‘Select by expression’ button in QGIS (the one that has a Greek-style ‘E’ and the yellow selection square), select and extract the M-Roads and A-Roads to two separate shapefiles. Note: make sure that when you use ‘Save as’ to save the selected road type you also remember to check ‘Save selected features’ as well!

You can find the type of road in the ‘class’ attribute but, be warned, both select-and-extract operations may take some time as we still have to start with the very large file and there are still a lot of A roads in Britain.
Once you’ve added the slightly smaller road files to QGIS you can delete the full open roads layer from QGIS and from your desktop to free up space. 

You can also remove the annoying way that line segments are shown by default (i.e. you see the ‘joins’ between segements) as follows:
* Right-click on the layer to gain access to the Layer Properties.
* Select the Style tab.
* Click on the ‘Advanced’ drop-down menu towards the lower right.
* Select ‘Symbol levels’ and, in the resulting pop-up, tick the ‘Enable symbol levels’ checkbox so that symbol levels are activated. 

Sorted.

## Obtaining Defra Land Use Classification data

And finally, the full Agricultural Land Classification data from Defra (Department for the Environment, Food and Rural Affairs) is available from their [Dataset Download Summary](http://magic.defra.gov.uk/Dataset_Download_Summary.htm) page.

From this you’ll actually be guided to the Environment Agency’s web site which has slightly newer data (and more formats), but I want you to note on both pages that there are a lot of other interesting data sets here that you might want to investigate if you’re planning a data-intensive IGS (especially one that looks at physical geography or human/physical interactions), or if you’ve not yet finished your work for the QGIS intensive.
